### Rag Fusion
Unlike naive union of Multiqueryretriever, rag-fusion has a ranker at the end of multi-query layer that ranks the relevance of contexts and consumes them  When multiple queries are passed through the retriever, it generates a list of relevant documents for each query. By ranking the order of relevance of each context, we focus on the most relevant pieces of information first and less relevant information later. The calculation is in such a way that even the lower ranked documents are taken into consideration. This maximizes the contextual information passed on to the LLM.

In [2]:
from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os 


load_dotenv()
os.getenv("GOOGLE_API_KEY") 
my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [42]:
docs = {
    "doc1": "Feminism and economic empowerment.",
    "doc2": "Family life disintegration due to Feminism",
    "doc3": "Divorce & Feminism: A social perspective.",
    "doc4": "Conflict Resolution via Counselling",
    "doc5": "Policy changes needed to combat family disintegration and divorces.",
    "doc6": "Divorces and its impact on society.",
    "doc7": "Feminism: Laws and Policies.",
    "doc8": "Divorce: A subset of Feminism.",
    "doc9": "How Feminism affects daily life of womenkind?",
    "doc10": "The history of Feminism.",
}

In [43]:
from langchain.schema import Document
# Convert documents to LangChain Document format
docs = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in docs.items()]


In [44]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

## Call Embedding Model
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")


vectorstore = Chroma.from_documents(documents=docs, 
                                    embedding=embedding)

retriever = vectorstore.as_retriever()

##### Defining the Query Generator

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM
llm = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [46]:
from langchain import hub

prompt = hub.pull("langchain-ai/rag-fusion-query-generation")
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {original_query}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='OUTPUT (4 queries):'))]


In [47]:
generate_queries = (
    prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
)

##### Full chain
We can now put it all together and define the full chain. This chain:

1. Generates a bunch of queries
2. Looks up each query in the retriever
3. Joins all the results together using <b> Reciprocal Rank Fusion </b>

In [51]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=2):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion


In [56]:
original_query = " Feminism and Policies" # play around with feminism and women rights and view

docs = retrieval_chain_rag_fusion.invoke({"original_query": original_query})
docs

[(Document(page_content='Women Rights: Laws and Policies.', metadata={'id': 'doc7'}),
  2.1999999999999997),
 (Document(page_content='Policy changes needed to combat family disintegration and divorces.', metadata={'id': 'doc5'}),
  2.1666666666666665),
 (Document(page_content='Feminism: Laws and Policies.', metadata={'id': 'doc7'}),
  2.0),
 (Document(page_content='How Feminism affects daily life of womenkind?', metadata={'id': 'doc9'}),
  1.0666666666666667),
 (Document(page_content='How Women Rights affects daily life of womenkind?', metadata={'id': 'doc9'}),
  0.8500000000000001),
 (Document(page_content='Feminism and economic empowerment.', metadata={'id': 'doc1'}),
  0.7)]